In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.imputation import mice
from sklearn.utils import shuffle
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import numpy as np
from sklearn.preprocessing import RobustScaler
import time

# 데이터 불러오기 (전처리 완료)

In [3]:
sample = pd.read_csv('sample_submission.csv')
train = pd.read_csv('data_p.csv')
test = pd.read_csv('test_p.csv')

# 최종 예측

In [11]:
# X_train should be your feature matrix (excluding the target variable), and y_train your target variable
X_train = train.drop('Target', axis=1)
y_train = train['Target']

# X_test should be your feature matrix for the test set
X_test = test

start_time = time.time()

train_data = lgb.Dataset(X_train, label=y_train)

# Define LightGBM parameters
params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'boosting_type': 'goss',
    'num_leaves': 28, # You can experiment with this value
    'learning_rate': 0.01,
    'feature_fraction': 0.33,
    'bagging_fraction': 1,
    'bagging_freq': 5,
    'min_data_in_leaf': 40,
    'scale_pos_weight': 33  # Adjust based on your positive rate (1 / positive_rate - 1)
}

# Train the model
num_round = 300
bst = lgb.train(params, train_data, num_round)

training_time = time.time() - start_time
print(f"Training Time: {training_time} seconds")

# Measure prediction time
start_time = time.time()

y_pred = bst.predict(X_test, num_iteration=bst.best_iteration)

prediction_time = time.time() - start_time
print(f"Prediction Time: {prediction_time} seconds")

[LightGBM] [Warning] Found boosting=goss. For backwards compatibility reasons, LightGBM interprets this as boosting=gbdt, data_sample_strategy=goss.To suppress this warning, set data_sample_strategy=goss instead.
[LightGBM] [Warning] Found boosting=goss. For backwards compatibility reasons, LightGBM interprets this as boosting=gbdt, data_sample_strategy=goss.To suppress this warning, set data_sample_strategy=goss instead.
[LightGBM] [Info] Number of positive: 176, number of negative: 5279
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000244 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2805
[LightGBM] [Info] Number of data points in the train set: 5455, number of used features: 11
[LightGBM] [Info] Using GOSS
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.032264 -> initscore=-3.401008
[LightGBM] [Info] Start training from score -3.401008
Training Time: 0.1449437141418457 seconds
Prediction Ti

101

In [4]:
# Assuming you have a binary classification problem, you can convert probabilities to binary predictions
y_pred = (y_pred > 0.5).astype(int)
sample['Target'] = y_pred
sample.to_csv("final_submission.csv", index=False)